In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/3.vehicle'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            data = {'func': 'getbuyersguide',
                    'payload': json.dumps({'partData': {'listing_data_essential': {'parttype': crawler_series['Type Code'],
                                                                                   'partkey': crawler_series['Part Code']},
                                                        'listing_data_supplemental': {'partnumber': crawler_series['Part Number'],
                                                                                      'catalogname': crawler_series['Manufacturer']}}}),
                    'api_json_request': '1'}

            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_check = crawler_series['Url']

                    resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                         data=data,
                                         headers=get_header(),
                                         proxies=get_proxy(),
                                         timeout=(5, 15))

                    if resp.status_code == 200:
                        dict_ = resp.json()

                        if 'redirect_to_url' not in dict_ and dict_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in dict_:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(dict_['buyersguidepieces']['body'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_tr = html.xpath('//tr')

            # = = = = = = = = = = = = = = =

            if not list_tr and not (html.xpath('//p/text()') and html.xpath('//p/text()')[0].strip() == 'No applications found.'):
                raise

            # = = = = = = = = = = = = = = =

            list_dict_correct_temp = []
            for i, tr in enumerate(list_tr):
                list_text = [text.strip() for text in tr.xpath('./td/text()')]

                if len(list_text) == 3:
                    list_year = list_text[2].split('-')
                    year_begin, year_end = int(list_year[0]), int(list_year[-1])
                    list_dict_correct_temp_temp = [{'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC'],
                                                    'Vehicle No': i + 1,
                                                    'Make': list_text[0],
                                                    'Model': list_text[1],
                                                    'Year': year} for year in range(year_begin, year_end + 1)]
                elif len(list_text) == 2:
                    list_year = list_text[1].split('-')
                    year_begin, year_end = int(list_year[0]), int(list_year[-1])
                    list_dict_correct_temp_temp = [{'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC'],
                                                    'Vehicle No': i + 1,
                                                    'Make': list_text[0],
                                                    'Year': year} for year in range(year_begin, year_end + 1)]
                else:
                    print('fuck')

                list_dict_correct_temp.extend(list_dict_correct_temp_temp)

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Check_Url'] = url_check

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 0 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_check}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['JOIN_MPNTCPC', 'Vehicle No', 'Year'],
                                                                 ascending=[True, True, False],
                                                                 ignore_index=True)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

总数量：10

[True] - [请求1次] - [剩余9条] - [crawler_1（50.00%）] - [2024/10/18 10:10:15] - 3. https://www.rockauto.com/en/parts/melling,30420S,timing+chain+&+component+kit,5756

[True] - [请求1次] - [剩余8条] - [crawler_1（50.00%）] - [2024/10/18 10:10:15] - 4. https://www.rockauto.com/en/parts/melling,30430S,timing+chain+&+component+kit,5756

[True] - [请求1次] - [剩余7条] - [crawler_1（50.00%）] - [2024/10/18 10:10:15] - 2. https://www.rockauto.com/en/parts/melling,30250SX,timing+chain+&+component+kit,5756

[True] - [请求1次] - [剩余6条] - [crawler_1（50.00%）] - [2024/10/18 10:10:16] - 5. https://www.rockauto.com/en/parts/melling,30460S,timing+chain+&+component+kit,5756

[True] - [请求1次] - [剩余5条] - [crawler_1（50.00%）] - [2024/10/18 10:10:16] - 7. https://www.rockauto.com/en/parts/melling,30460SB,timing+chain+&+component+kit,5756

[True] - [请求1次] - [剩余4条] - [crawler_1（50.00%）] - [2024/10/18 10:10:16] - 8. https://www.rockauto.com/en/parts/melling,30470S,timing+chain+&+component+kit,5756

[True] - [请求1次] - [剩余3条] - [cr

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3215.26it/s]

Done ~
